In [23]:
# from openai import OpenAI
import toml
# from ollama import chat
# from ollama import ChatResponse
from ollama import Client
from langchain_core.prompts import PromptTemplate
import json
from langchain_ollama.llms import OllamaLLM
from langchain.chains.llm import LLMChain
from duckduckgo_search import DDGS


In [2]:

# Load a TOML file
with open('../secrets.toml', 'r') as f:
    openai_key = toml.load(f).get('api_keys').get('openai')

assert openai_key, "OpenAI API key not found in secrets.toml"


In [3]:
class LLMModel:
    def __init__(self, model_name: str, run_local: bool = True, system_prompt: str = "you are a helpful assistant"):
        self.run_local = run_local
        self.model_name = model_name
        self.system_prompt = system_prompt
        self.messages = [{"role": "system", "content": self.system_prompt}]
        
        if self.run_local:
            self.client = Client(
                host='http://localhost:11434',
                # headers={'x-some-header': 'some-value'}
                )

    def add_message(self, role: str, content: str) -> bool:
        if role not in ["system", "user", "assistant"]:
            raise ValueError("Role must be 'system', 'user', or 'assistant'")
        self.messages.append({"role": role, "content": content})
        return True

    def clear_messages(self) -> bool:
        self.messages = []
        return True

    def local_generate(self, prompt: str, max_tokens: int = 1500, **kwargs) -> str:
        self.messages.append({"role": "user", "content": prompt})
        response = self.client.chat(model=self.model_name, messages=self.messages, **kwargs)
        self.messages.append({"role": "assistant", "content": response.message.content.strip()})
        return response.message.content.strip()
    
    # question: how do I set max token in ollama python api?
    


In [4]:
model = LLMModel(model_name="llama3.2:latest", run_local=True)

In [5]:
# model.local_generate("What is the capital of India?")

In [6]:
# model.local_generate("What is the weather like there?")

In [7]:
# model.local_generate("What is the best month to visit?")

In [8]:
# model.local_generate("What is the best month to visit? Just give me the month name, no other text.")

In [9]:
# model.messages

In [ ]:
# model.clear_messages()

True

In [35]:
topic = "The Emergency in India (1975-1977)"
number_of_subtopics = 5


In [ ]:
subtopics_template = PromptTemplate.from_template(
    """Generate a list of {number_of_subtopics} subheadings for a highschool essay on {topic}.
    Order the subheadings such that they have a logical sequence.
    Each subheading should have a one-line discription.
    Provide output in json format as follows:

    {{<heading1>:<description1>,
    <heading2>:<description2>,...}}"""
)

In [ ]:
subtopics_prompt = subtopics_template.format(topic=topic, number_of_subtopics=number_of_subtopics)
subtopics_prompt

'Generate a list of 5 subheadings for a highschool essay on The Emergency in India (1975-1977).\n    Order the subheadings such that they have a logical sequence.\n    Each subheading should have a one-line discription.\n    Provide output in json format as follows:\n\n    {<heading1>:<description1>,\n    <heading2>:<description2>,...}'

In [47]:
topics = model.local_generate(subtopics_prompt, format='json')
topics = json.loads(topics)
topics

{'Causes_of_the_emergency': 'Analyze the historical events that led to the declaration of emergency by Prime Minister Indira Gandhi in 1975.',
 'Effects_on_indian_politics': "Explain how the Emergency affected India's political landscape and its impact on the country's democratic institutions.",
 'Censorship_and_media_control': 'Discuss the measures taken by the government to control media coverage during the Emergency period, including censorship and suppression of opposition voices.',
 'Dissolution_of_constituent_assemblies': 'Describe the dissolution of the constituent assemblies at the Centre and the States by the President on June 25, 1975, marking a significant turning point in the Emergency',
 'Legacy_of_the_emergency_and_reforms': "Assess the lasting impact of the Emergency period on India's politics, economy, and society, including the subsequent reforms introduced by Prime Minister Rajiv Gandhi."}

In [48]:
for k,v in topics.items():
    print(k)
    print(v)

Causes_of_the_emergency
Analyze the historical events that led to the declaration of emergency by Prime Minister Indira Gandhi in 1975.
Effects_on_indian_politics
Explain how the Emergency affected India's political landscape and its impact on the country's democratic institutions.
Censorship_and_media_control
Discuss the measures taken by the government to control media coverage during the Emergency period, including censorship and suppression of opposition voices.
Dissolution_of_constituent_assemblies
Describe the dissolution of the constituent assemblies at the Centre and the States by the President on June 25, 1975, marking a significant turning point in the Emergency
Legacy_of_the_emergency_and_reforms
Assess the lasting impact of the Emergency period on India's politics, economy, and society, including the subsequent reforms introduced by Prime Minister Rajiv Gandhi.


In [50]:
def generate_subtopics(topic: str, number_of_subtopics: int, model: LLMModel) -> dict:
    subtopics_template = PromptTemplate.from_template(
    """Generate a list of {number_of_subtopics} subheadings for a highschool essay on {topic}.
    Order the subheadings such that they have a logical sequence.
    Each subheading should have a one-line discription.
    Provide output in json format as follows:

    {{<heading1>:<description1>,
    <heading2>:<description2>,...}}"""
    )
    
    subtopics_prompt = subtopics_template.format(topic=topic, number_of_subtopics=number_of_subtopics)

    topics = model.local_generate(subtopics_prompt, format='json')
    topics = json.loads(topics)
    return topics

In [51]:
generate_subtopics("History of the Internet", 6, model)

{'Early Beginnings': 'Discuss the origins of the internet, from its development in the 1960s by the United States Department of Defense to its first functional networks.',
 'ARPANET and Network Expansion': 'Explain how ARPANET, the first operational packet switching network, expanded into a global network through various government and private initiatives.',
 'TCP/IP and the Internet Protocol': 'Describe the development of TCP/IP (Transmission Control Protocol/Internet Protocol), which became the standard communication protocol for the internet.',
 'World Wide Web and HTML': 'Introduce the World Wide Web and Hypertext Markup Language (HTML), technologies invented by Tim Berners-Lee that enabled web browsing and online content sharing.',
 'Internet Governance and Regulation': 'Analyze the evolution of internet governance, including the establishment of organizations such as ICANN (Internet Corporation for Assigned Names and Numbers) and efforts to regulate online content.',
 'Impact and

In [52]:
generate_subtopics("Evolution of Agentic AI", 10, model)

{'Introduction to Agentic AI': 'Introduce the concept of agentic AI, which refers to artificial intelligence that can make decisions and act independently.',
 'Early Foundations: Robotics and Simulation': 'Discuss the early foundations of agentic AI, including robotics and simulation research that laid the groundwork for more advanced systems.',
 'Machine Learning and Reinforcement Learning': 'Explain how machine learning and reinforcement learning techniques have been used to create agentic AI systems that can learn from experience and make decisions.',
 'Development of AlphaGo: A Key Breakthrough': 'Describe the development of AlphaGo, a computer program that defeated a human world champion in Go, marking a significant milestone in agentic AI research.',
 'Modern Agentic AI: Applications and Use Cases': 'Discuss various applications and use cases of modern agentic AI, including autonomous vehicles, drones, and smart home systems.',
 'Value Alignment and Ethics': 'Analyze the importan

In [46]:
def subtopic_chain() -> LLMChain:
    subtopics_template = PromptTemplate.from_template(
        """Generate a list of {number_of_subtopics} subheadings for a highschool essay on {topic}.
        Order the subheadings such that they have a logical sequence.
        Each subheading should be accompanied with a detailed discription on what the contents should be about.
        Provide output in json format as follows:

        {{'<heading1>':'<description1>',
        '<heading2>':'<description2>',...}}
        
        Only return the json object without any additional text."""
    )
    
    model = OllamaLLM(model="llama3.2:latest", run_local=True, format='json')

    return LLMChain(llm=model, prompt=subtopics_template)

In [52]:
chain = subtopic_chain()
subtopics = chain.run(topic="History of the Internet", number_of_subtopics=6)
subtopics = json.loads(subtopics)
subtopics

{'Early Beginnings of the Internet': "This section will explore the origins and early development of the internet, including key events such as ARPANET's creation in 1969 and the Network Control Protocol (NCP) that enabled communication between different networks.",
 'The Dawn of the World Wide Web': 'This subsection will discuss how Tim Berners-Lee invented the world wide web in 1989, and how it revolutionized the way people access and share information online.',
 'Internet Expansion and Growth': 'This section will cover the rapid expansion and growth of the internet in the 1990s, including the development of dial-up connections, the rise of email, and the emergence of online communities.',
 "The Internet's Impact on Society": 'This subsection will examine how the internet has transformed the way people communicate, access information, and conduct business, as well as its impact on education, healthcare, and other areas of society.',
 'Security Concerns and Cybercrime': 'This section 

In [59]:
def search_query_chain() -> LLMChain:
    search_query_template = PromptTemplate.from_template(
        """Generate a search query for Duckduckgo on the topic: {search_details}.
        The query should be concise and relevant to the topic.
        Return only the search query without any additional text."""
    )
    
    model = OllamaLLM(model="llama3.2:latest", run_local=True)

    return LLMChain(llm=model, prompt=search_query_template)

In [57]:
def get_search_queries(subtopics: dict, search_query_chain: LLMChain) -> dict:
    search_queries = {}
    for subtopic, desc in subtopics.items():
        search_query = search_query_chain.run(search_details=desc)
        search_queries[subtopic] = search_query
    return search_queries

In [60]:
search_query_chain = search_query_chain()
search_queries = get_search_queries(subtopics, search_query_chain)
search_queries

{'Early Beginnings of the Internet': '"ARPANET creation and Network Control Protocol early development"',
 'The Dawn of the World Wide Web': '"Tim Berners-Lee invention of the World Wide Web"',
 'Internet Expansion and Growth': '"Internet 1990s expansion and growth"',
 "The Internet's Impact on Society": '"Internet\'s impact on communication, information, business, education, healthcare, and society"',
 'Security Concerns and Cybercrime': '"security measures against hacking and cybercrime"',
 'The Modern Internet: Challenges and Future Directions': '"internet trends 2023 regulation blockchain misinformation"'}

In [ ]:
# def information_chain() -> LLMChain:
#     information_template = PromptTemplate.from_template(
#         """Generate details for the"""
#     )
    
#     model = OllamaLLM(model="llama3.2:latest", run_local=True)

#     return LLMChain(llm=model, prompt=information_template)

In [ ]:
# results = DDGS().text("the emergency")

In [ ]:
# import requests
# from bs4 import BeautifulSoup

In [ ]:
# query = "what is the best search engine"
# url = "https://www.duckduckgo.com/?t=h_&q=" + query.replace(" ", "+") +"&t=h_&ia=web"
# response = requests.get(url)
# response.raise_for_status()
# soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
# response.content

b'<!DOCTYPE html><html lang="en-US" class="no-js has-zcm  no-theme is-link-style-exp is-link-order-exp is-link-breadcrumb-exp is-related-search-exp is-vertical-tabs-exp  "><head><meta name="description" content="DuckDuckGo. Privacy, Simplified."><meta http-equiv="content-type" content="text/html; charset=utf-8"><title>what is the best search engine at DuckDuckGo</title><meta name="apple-itunes-app" content="app-id=663592361, app-argument=https://duckduckgo.com/?t=h_&q=what+is+the+best+search+engine&t=h_&ia=web&smartbanner=1"><link rel="stylesheet" href="/dist/s.f7c43807461107926ad2.css" type="text/css"><link rel="stylesheet" href="/dist/r.afc03c5e103c89381710.css" type="text/css"><link rel="stylesheet" href="/dist/wpl.main.40a6dca6d087eb867eed.css" type="text/css" data-handle-css-custom-properties="true"><link rel="stylesheet" href="/dist/wpl.vendors.15a9724ad11a243a515a.css" type="text/css" data-handle-css-custom-properties="true"><meta name="robots" content="noindex,nofollow"><meta n

In [ ]:
# def get_

In [ ]:
# chain for subtopics - chain for search queries - chain for information retrieval from search results - chain for summarization